In [11]:
import os

In [12]:
os.chdir("../")

In [13]:
%pwd

'd:\\Projects\\Chicken Disease Classification'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallbacksConfig:
    root_dir : Path 
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path

In [15]:
from CDC.constants import *
from CDC.utils.helper_functions import read_yaml, create_directories

In [23]:
from CDC.constants import *
from CDC.utils.helper_functions import read_yaml, create_directories
from CDC.entity.config_entity import DataIngestionConfig, BaseModelConfig

from box import ConfigBox

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> ConfigBox:
        config = self.config.data_ingestion
        create_directories([config.root_dir])    

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            unzip_dir = config.unzip_dir,
            local_data_file = config.local_data_file,
            source_url = config.source_URL
        )

        return data_ingestion_config
    
    def get_base_model_config(self) -> BaseModelConfig:
        config = self.config.base_model
        create_directories([config.root_dir])
        base_model_config = BaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.INPUT_SHAPE,
            params_batch_size = self.params.BATCH_SIZE,
            params_epochs = self.params.EPOCHS,
            params_learning_rate = self.params.LEARNING_RATE,
            params_classes = self.params.CLASSES,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS
        )
        
        return base_model_config

    def get_callbacks_config(self) -> CallbacksConfig:
        config = self.config.callbacks
        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([config.tensorboard_root_log_dir,
                            model_checkpoint_dir])
        callbacks_config = CallbacksConfig(
            root_dir = Path(config.root_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir)
        )
        return callbacks_config

    


In [27]:

import time
import tensorflow as tf
class CallBacks:
    def __init__(self,config: CallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
                    self._create_tb_callbacks,
                    self._create_ckpt_callbacks
        ]








In [28]:
try:
    configuration = ConfigurationManager()
    print("1")
    callbacks_config = configuration.get_callbacks_config()
    print("2")
    callbacks = CallBacks(callbacks_config)
    print("3")
    callbacks_list = callbacks.get_tb_ckpt_callbacks()
    print("4")
except Exception as e:
    print(e)

[2023-06-21 17:48:32,214: INFO: helper_functions: yaml file config\config.yaml uploaded successfully]
[2023-06-21 17:48:32,222: INFO: helper_functions: yaml file params.yaml uploaded successfully]
[2023-06-21 17:48:32,226: INFO: helper_functions: created directory at: artifacts]
1
[2023-06-21 17:48:32,231: INFO: helper_functions: created directory at: artifacts/callbacks/tensorboard_log_dir]
[2023-06-21 17:48:32,233: INFO: helper_functions: created directory at: artifacts/callbacks/checkpoint_dir]
2
3
4
